# Bundle Recommendation – Prediction / Inference

## Objective
This notebook demonstrates how a user or downstream system
can interact with the trained product bundle recommender.

It simulates:
- User input (product selection)
- Model inference
- Human-readable output

This notebook performs **no training or evaluation**.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
BASE_DIR = Path().resolve().parent
FEATURE_PATH = BASE_DIR / "data" / "features"

co_df = pd.read_parquet(FEATURE_PATH / "weighted_co_occurrence.parquet")
popularity_df = pd.read_parquet(FEATURE_PATH / "product_popularity.parquet")

print("Artifacts loaded successfully.")

Artifacts loaded successfully.


In [3]:
# Popularity lookup
popularity_map = dict(
    zip(popularity_df["StockCode"], popularity_df["popularity"])
)

# Co-occurrence lookup
co_map = {}

for _, row in co_df.iterrows():
    a, b, score = row["product_a"], row["product_b"], row["weighted_score"]
    
    co_map.setdefault(a, {})[b] = score
    co_map.setdefault(b, {})[a] = score

print("Lookup structures ready.")

Lookup structures ready.


In [4]:
CLEAN_FILE = BASE_DIR / "data" / "processed" / "clean_transactions.parquet"
clean_df = pd.read_parquet(CLEAN_FILE)

product_name_map = (
    clean_df.groupby("StockCode")["Description"]
    .agg(lambda x: x.mode().iloc[0])
    .to_dict()
)

print("Product name mapping loaded.")

Product name mapping loaded.


In [5]:
def score_product(target_product, candidate_product):
    co_score = co_map.get(target_product, {}).get(candidate_product, 0)
    popularity = popularity_map.get(candidate_product, 1)
    
    return co_score / np.log1p(popularity)

In [6]:
def recommend_bundle(product_id, top_k=5):
    """
    Recommend complementary products for a given product.
    """
    if product_id not in co_map:
        return []
    
    scored_candidates = [
        (prod, score_product(product_id, prod))
        for prod in co_map[product_id]
    ]
    
    scored_candidates.sort(key=lambda x: x[1], reverse=True)
    
    return scored_candidates[:top_k]

In [7]:
def cold_start_recommendation(top_k=5):
    """
    Fallback recommendations for unseen products.
    """
    popular = sorted(
        popularity_map.items(),
        key=lambda x: x[1],
        reverse=True
    )
    return popular[:top_k]

In [8]:
def predict_bundle(product_id, top_k=5):
    """
    Unified interface for bundle prediction.
    """
    if product_id in co_map:
        return recommend_bundle(product_id, top_k)
    else:
        return cold_start_recommendation(top_k)

## User Input

The user provides a product (StockCode).
The system returns a list of complementary products.

In [9]:
# Pick an example product from the dataset
example_product_id = list(co_map.keys())[0]

example_product_id, product_name_map.get(example_product_id)

('21730', 'GLASS STAR FROSTED T-LIGHT HOLDER')

In [10]:
recommendations = predict_bundle(example_product_id, top_k=5)
recommendations

[('71477', np.float64(1.7948887125479207)),
 ('85123A', np.float64(1.573257266242966)),
 ('23313', np.float64(1.3029139004888437)),
 ('23322', np.float64(1.2740848082979412)),
 ('23355', np.float64(1.238314017860616))]

In [11]:
output = [
    {
        "Recommended Product": product_name_map.get(prod, prod),
        "Score": round(score, 4)
    }
    for prod, score in recommendations
]

pd.DataFrame(output)

,Recommended Product,Score
0,COLOUR GLASS. STAR T-LIGHT HOLDER,1.7949
1,WHITE HANGING HEART T-LIGHT HOLDER,1.5733
2,VINTAGE CHRISTMAS BUNTING,1.3029
3,LARGE WHITE HEART OF WICKER,1.2741
4,HOT WATER BOTTLE KEEP CALM,1.2383


## Cold-Start Example

Simulate a product that does not exist in historical data.

In [12]:
cold_start_product = "UNKNOWN_PRODUCT_123"

cold_recommendations = predict_bundle(cold_start_product, top_k=5)

pd.DataFrame([
    {
        "Recommended Product": product_name_map.get(prod, prod),
        "Score": round(score, 4)
    }
    for prod, score in cold_recommendations
])

,Recommended Product,Score
0,"PAPER CRAFT , LITTLE BIRDIE",80995
1,MEDIUM CERAMIC TOP STORAGE JAR,78033
2,POPCORN HOLDER,56921
3,WORLD WAR 2 GLIDERS ASSTD DESIGNS,55047
4,JUMBO BAG RED RETROSPOT,48474


## User Experience Summary

- Input: Product selected by user
- Output: Complementary products ranked by relevance
- Fallback: Popular products for unseen items

This notebook demonstrates how the recommendation system
would behave in a real product or API setting.